In [14]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

data = pd.read_csv('Datasets/classified_data1.csv',error_bad_lines=False, index_col=False, dtype='unicode', nrows=100000)
#data = data.query("state='Alabama'")

data.sort_values("likeCount", inplace = True)
data["likeCount"] = pd.to_numeric(data["likeCount"])
data["replyCount"] = pd.to_numeric(data["replyCount"])
data["retweetCount"] = pd.to_numeric(data["retweetCount"])
app = JupyterDash(__name__)

available_indicators = ["date","likeCount","replyCount","retweetCount","start date","sentiment"]
# add filter by state

minimum,maximum = 0, 3000



In [15]:


app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id="xaxis-column",
                options = [{'label': i, 'value': i} for i in available_indicators],
                value="likeCount"
            ),
            #Radio item functionality not implemented yet
            dcc.RadioItems(
                id="xaxis-type",
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            
            )
            

        ]),
        
        html.Div([
            dcc.Dropdown(
                id="yaxis-column",
                options = [{'label': i, 'value': i} for i in available_indicators],
                value="likeCount"
            ),
            #Radio item functionality not implemented yet
            dcc.RadioItems(
                id="yaxis-type",
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            
            )
            

        ])
    ]),
    
    
    
    dcc.Graph(id="over-time-scatter"),
    html.P("Like Slider"),
    dcc.RangeSlider(
        id="x-range-slider",
        min=minimum, max = maximum, step = 1,
        marks={minimum: str(minimum), maximum: str(maximum)},
        value=[minimum,maximum]
    ),
    
    ]   
)

@app.callback(
    Output("over-time-scatter", "figure"),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    [Input("x-range-slider", "value")])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type, slider_range):
    #low, high = slider_range
    #low, high = [0,1000]
    #mask = (data['likeCount'] > low) & (data['likeCount'] < high)
    fig = px.scatter(
        data, x=xaxis_column_name, y=yaxis_column_name)
    return fig


'''low, high = slider_range
    #low, high = [0,1000]
    mask = (data['likeCount'] > low) & (data['likeCount'] < high)
    fig = px.scatter(
        data[mask], x="date", y="likeCount")'''

'low, high = slider_range\n    #low, high = [0,1000]\n    mask = (data[\'likeCount\'] > low) & (data[\'likeCount\'] < high)\n    fig = px.scatter(\n        data[mask], x="date", y="likeCount")'

In [16]:
app.run_server(mode='inline', port=1002)